In [1]:
!pip install tensorboard
!pip install scipy


[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from tqdm import tqdm
from datetime import datetime
import numpy as np
import torch
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import os

In [3]:
class AlexNet(torch.nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        # Convolutional layers
        self.conv1 = torch.nn.Conv2d(3, 96, 11, stride=4)
        self.conv2 = torch.nn.Conv2d(96, 256, 5, padding=2)
        self.conv3 = torch.nn.Conv2d(256, 384, 3, padding=1)
        self.conv4 = torch.nn.Conv2d(384, 384, 3, padding=1)
        self.conv5 = torch.nn.Conv2d(384, 256, 3, padding=1)

        # Fully connected layers
        self.fc1 = torch.nn.Linear(9216, 4096)
        self.fc2 = torch.nn.Linear(4096, 4096)
        self.fc3 = torch.nn.Linear(4096, num_classes)

        # Initializatin of the weights
        torch.nn.init.normal_(self.conv1.weight, mean=0.0, std=0.01)
        torch.nn.init.normal_(self.conv2.weight, mean=0.0, std=0.01)
        torch.nn.init.normal_(self.conv3.weight, mean=0.0, std=0.01)
        torch.nn.init.normal_(self.conv4.weight, mean=0.0, std=0.01)
        torch.nn.init.normal_(self.conv5.weight, mean=0.0, std=0.01)
        torch.nn.init.normal_(self.fc1.weight, mean=0.0, std=0.01)
        torch.nn.init.normal_(self.fc2.weight, mean=0.0, std=0.01)
        torch.nn.init.normal_(self.fc3.weight, mean=0.0, std=0.01)

        # Initializatin of the bias
        torch.nn.init.constant_(self.conv2.bias, 1.0)
        torch.nn.init.constant_(self.conv4.bias, 1.0)
        torch.nn.init.constant_(self.conv5.bias, 1.0)
        torch.nn.init.constant_(self.conv1.bias, 0.0)
        torch.nn.init.constant_(self.conv3.bias, 0.0)
        torch.nn.init.constant_(self.fc1.bias, 0.0)
        torch.nn.init.constant_(self.fc2.bias, 0.0)
        torch.nn.init.constant_(self.fc3.bias, 0.0)

        # The definition of alpha in the paper and pytorch are slightly different
        self.lrn = torch.nn.LocalResponseNorm(size=5, alpha=5e-4, beta=0.75, k=2.0)
        self.relu = torch.nn.ReLU(inplace=True)
        self.pool = torch.nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, x):
        # Conv1 -> ReLU -> Pool -> LRN
        x = self.conv1(x)
        x = self.relu(x)
        x = self.lrn(x)
        x = self.pool(x)
        
        # Conv2 -> ReLU -> Pool -> LRN
        x = self.conv2(x)
        x = self.relu(x)
        x = self.lrn(x)
        x = self.pool(x)

        # Conv3 -> ReLU
        x = self.conv3(x)
        x = self.relu(x)
        
        # Conv4 -> ReLU
        x = self.conv4(x)
        x = self.relu(x)
        
        # Conv5 -> ReLU -> Pool
        x = self.conv5(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Flatten
        x = torch.flatten(x, 1)
        
        # FC1 -> ReLU -> Dropout
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # FC2 -> ReLU -> Dropout
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # FC3
        x = self.fc3(x)
        return x

In [4]:
# transpose is use to meet the shape of the tensor C x H x W rather than H x W x C
mean_pixels = np.float32(np.load('mean_pixels.npy').transpose(2, 0, 1))

def meanSubstraction(x):
    return x - mean_pixels

def toTensorNoScaling(x):
    return torch.from_numpy(np.array(x).transpose(2, 0, 1))

def PCAColorAugmentation(img, std=0.1, eps=1e-6):
    # Convert PIL image to numpy array
    img = np.array(img)
    img_shape = img.shape  # Original shape (H, W, C)
    # Flatten to (H*W, 3) for RGB channels
    img_flat = img.reshape(-1, 3)
    
    # Compute covariance matrix between channels (3x3)
    img_cov = np.cov(img_flat, rowvar=False)
    # Get eigenvalues and eigenvectors (sorted in ascending order)
    eig_vals, eig_vecs = np.linalg.eigh(img_cov)
    eig_vals = eig_vals[::-1]  # Descending order
    eig_vecs = eig_vecs[:, ::-1]
    eig_vals = np.maximum(eig_vals, eps)
    # Generate random alphas from N(0, std)
    alpha = np.random.normal(0, std, 3)
    
    # Compute correction using sqrt of eigenvalues (standard deviations)
    correction = eig_vecs @ (alpha * np.sqrt(eig_vals))
    # Reshape correction for broadcasting (1, 3) over all pixels
    correction = correction.reshape(1, 3)
    # Apply correction and clip values
    new_img_flat = img_flat + correction
    new_img_flat = np.clip(new_img_flat, 0, 255).astype(np.float32)
    # Reshape to original dimensions, transpose to CxHxW and put between [0-1]
    new_img = new_img_flat.reshape(img_shape).transpose(2, 0, 1)
    
    return torch.from_numpy(new_img)  # Shape (C, H, W)

# Image transformation
transform_train = transforms.Compose([
    # Apply PCA color transformation
    transforms.Lambda(PCAColorAugmentation),
    # Remove mean
    transforms.Lambda(meanSubstraction),
    # 227x227 random crop 
    transforms.RandomCrop(227),
    # Horizontal reflection with p=0.5
    transforms.RandomHorizontalFlip(p=0.5),
])

transform_val = transforms.Compose([
    # Transform to tensor without scaling
    transforms.Lambda(toTensorNoScaling),
    # Remove mean
    transforms.Lambda(meanSubstraction),
    # 227x227 random crop 
    transforms.FiveCrop(227),
])

In [5]:
def train_one_epoch(epoch_index, writer, model, training_dataloader, optimizer, loss_fn, device):
    """
    Trains AlexNet for one epoch, tracking loss, top-1 and top-5 error rates.
    
    Args:
        epoch_index (int): Current epoch number
        writer: TensorBoard writer object
        model: AlexNet model instance
        training_dataloader: PyTorch dataloader for training data
        optimizer: SGD optimizer instance
        loss_fn: CrossEntropyLoss instance
        device: Device to train on (cuda/cpu)
    
    Returns:
        tuple: (average_loss, average_accuracy) for the epoch
    """
    total_loss = 0.0
    running_loss = 0.0
    total_top1_error = 0.0
    total_top5_error = 0.0
    running_top1_error = 0.0
    running_top5_error = 0.0
    
    # Ensure model is in training mode
    model.train()
    
    for i, data in enumerate(tqdm(training_dataloader)):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate top-1 and top-5 error rates
        _, top5_preds = outputs.topk(5, dim=1, largest=True, sorted=True)
        correct_top1 = top5_preds[:, 0] == labels
        correct_top5 = top5_preds.eq(labels.view(-1, 1)).sum(dim=1) > 0
        top1_error = 1 - correct_top1.sum().item() / labels.size(0)
        top5_error = 1 - correct_top5.sum().item() / labels.size(0)
        
        # Update running statistics
        running_loss += loss.item()
        running_top1_error += top1_error
        running_top5_error += top5_error

        # Update total statistics
        total_loss += loss.item() 
        total_top1_error += top1_error
        total_top5_error += top5_error
        
        # Log every 1000 batches
        if i % 1000 == 999:
            avg_running_loss = running_loss / 1000
            avg_top1_error = 100.0 * running_top1_error / 1000
            avg_top5_error = 100.0 * running_top5_error / 1000
                        
            # Log to TensorBoard
            tb_x = epoch_index * len(training_dataloader) + i + 1
            writer.add_scalar('Loss/train_step', avg_running_loss, tb_x)
            writer.add_scalar('Top-1 error/train_step', avg_top1_error, tb_x)
            writer.add_scalar('Top-5 error/train_step', avg_top5_error, tb_x)

            print(f'  Batch {tb_x} Loss: {avg_running_loss:.4f} Top-1 error rate: {avg_top1_error:.2f}% Top-5 error rate: {avg_top5_error:.2f}%')
            
            running_loss = 0.0
            running_top1_error = 0.0
            running_top5_error = 0.0
    
    # Calculate epoch-level metrics
    avg_epoch_loss = total_loss / len(training_dataloader)
    avg_top1_error = 100.0 * total_top1_error / len(training_dataloader)
    avg_top5_error = 100.0 * total_top5_error / len(training_dataloader)
    
    return avg_epoch_loss, avg_top1_error, avg_top5_error

In [6]:
def validate_one_epoch(epoch_index, writer, model, validation_dataloader, loss_fn, device):
    """
    Validates AlexNet for one epoch, tracking loss, top-1 and top-5 error rates.
    
    Args:
        epoch_index (int): Current epoch number
        writer: TensorBoard writer object
        model: AlexNet model instance
        validation_dataloader: PyTorch dataloader for validation data (with FiveCrop)
        loss_fn: CrossEntropyLoss instance
        device: Device to train on (cuda/cpu)
    
    Returns:
        tuple: (average_loss, average_top1_error, average_top5_error) for the epoch
    """
    total_loss = 0.0
    total_top1_error = 0.0
    total_top5_error = 0.0
    running_loss = 0.0
    running_top1_error = 0.0
    running_top5_error = 0.0
    
    # Ensure model is in evaluation mode
    model.eval()
    
    with torch.no_grad():  # Disable gradient computation
        for i, data in enumerate(tqdm(validation_dataloader, desc='Validation')):
            inputs, labels = data
            # Move each crop tensor in the inputs tuple to the device
            inputs = [inp.to(device) for inp in inputs]
            labels = labels.to(device)
            
            # Combine all 5 crops into a single batch dimension (5*B, C, H, W)
            all_crops = torch.cat(inputs, dim=0)
            
            # Forward pass through the model
            outputs = model(all_crops)
            
            # Reshape outputs to (5, B, num_classes) and average across crops
            outputs = outputs.view(5, -1, outputs.shape[-1])
            avg_outputs = outputs.mean(dim=0)  # Shape (B, num_classes)
            
            # Calculate loss using averaged outputs
            loss = loss_fn(avg_outputs, labels)
            
            # Calculate top-1 and top-5 error rates
            _, top5_preds = avg_outputs.topk(5, dim=1, largest=True, sorted=True)
            correct_top1 = top5_preds[:, 0] == labels
            correct_top5 = top5_preds.eq(labels.view(-1, 1)).any(dim=1)
            top1_error = 1 - correct_top1.float().mean().item()
            top5_error = 1 - correct_top5.float().mean().item()
            
            # Update running statistics
            running_loss += loss.item()
            running_top1_error += top1_error
            running_top5_error += top5_error

            # Update total statistics
            total_loss += loss.item()
            total_top1_error += top1_error
            total_top5_error += top5_error
            
            # Log every 100 batches
            if i % 100 == 99:
                avg_running_loss = running_loss / 100
                avg_top1_error = 100.0 * running_top1_error / 100
                avg_top5_error = 100.0 * running_top5_error / 100
                
                print(f'  Validation Batch {i + 1:5d} Loss: {avg_running_loss:.4f} Top-1 error: {avg_top1_error:.2f}% Top-5 error: {avg_top5_error:.2f}%')
                
                # Log to TensorBoard
                tb_x = epoch_index * len(validation_dataloader) + i + 1
                writer.add_scalar('Loss/val_step', avg_running_loss, tb_x)
                writer.add_scalar('Top-1 error/val_step', avg_top1_error, tb_x)
                writer.add_scalar('Top-5 error/val_step', avg_top5_error, tb_x)
                
                # Reset running statistics
                running_loss = 0.0
                running_top1_error = 0.0
                running_top5_error = 0.0
    
    # Calculate epoch-level metrics
    avg_epoch_loss = total_loss / len(validation_dataloader)
    avg_top1_error = 100.0 * total_top1_error / len(validation_dataloader)
    avg_top5_error = 100.0 * total_top5_error / len(validation_dataloader)
    
    return avg_epoch_loss, avg_top1_error, avg_top5_error

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Create model
alexnet = AlexNet(1000)
# First load
# pretrained_dict = torch.load('alexnette/model_20.pth', weights_only=True)
# alexnet_dict = alexnet.state_dict()
# pretrained_dict = {k: v for k, v in pretrained_dict.items() if k not in ['fc3.weight', 'fc3.bias']}
# alexnet_dict.update(pretrained_dict)
# alexnet.load_state_dict(alexnet_dict)
# Intermediate loads
# alexnet.load_state_dict(torch.load('models/in/model_58.pth', weights_only=True))
alexnet.to(device)

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=1000, bias=True)
  (lrn): LocalResponseNorm(5, alpha=0.0005, beta=0.75, k=2.0)
  (relu): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [8]:
filepath = '../imagenet'
print('Loading data...')
training_data = torchvision.datasets.ImageNet(filepath, split='train', transform=transform_train)
validation_data = torchvision.datasets.ImageNet(filepath, split='val', transform=transform_val)

# training_data = torchvision.datasets.Imagenette('.', 'train', transform=transform)
# validation_data = torchvision.datasets.Imagenette('.', 'val', transform=transform)

training_dataloader = torch.utils.data.DataLoader(
    training_data,
    batch_size=256,  
    shuffle=True,
    num_workers=16,  
    prefetch_factor=4,  
    persistent_workers=True,
    pin_memory=True,
)

validation_dataloader = torch.utils.data.DataLoader(
    validation_data,
    batch_size=128,  # I will get 5 out of each
    shuffle=False,
    num_workers=8,
    prefetch_factor=4,
    persistent_workers=True,
    pin_memory=True
)
print('Finished loading')

Loading data...
Finished loading


In [9]:
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/alexnet') # .format(timestamp)

loss_fn = torch.nn.CrossEntropyLoss()
# refer to https://github.com/dansuh17/alexnet-pytorch/blob/d0c1b1c52296ffcbecfbf5b17e1d1685b4ca6744/model.py#L142
# and https://discuss.pytorch.org/t/image-recognition-alexnet-training-loss-is-not-decreasing/66885/9 
# to justify the change
optimizer = torch.optim.SGD(alexnet.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
# optimizer = torch.optim.Adam(params=alexnet.parameters(), lr=1e-6, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

EPOCHS = 120
epoch_number = 0
best_vloss = 1000000.0
for epoch in range(EPOCHS):
    print(f'EPOCH {epoch_number + 1}:')

    # Training phase
    avg_train_loss, avg_train_top1_error, avg_train_top5_error = train_one_epoch(
        epoch_number,
        writer,
        alexnet,
        training_dataloader,
        optimizer,
        loss_fn,
        device,
    )

    # Validation phase
    avg_val_loss, avg_val_top1_error, avg_val_top5_error = validate_one_epoch(
        epoch_number,
        writer,
        alexnet,
        validation_dataloader,
        loss_fn,
        device
    )

    print(f'LOSS train {avg_train_loss:.4f} valid {avg_val_loss:.4f}')
    print(f'Top-1 Error train {avg_train_top1_error:.2f}% val {avg_val_top1_error:.2f}%')
    print(f'Top-5 Error train {avg_train_top5_error:.2f}% val {avg_val_top5_error:.2f}%')

    # Update scheduler
    scheduler.step()

    # Log epoch-level metrics
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training': avg_train_loss, 'Validation': avg_val_loss },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Top-1 Error',
                    { 'Training': avg_train_top1_error, 'Validation': avg_val_top1_error },
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Top-5 Error',
                    { 'Training': avg_train_top5_error, 'Validation': avg_val_top5_error },
                    epoch_number + 1)
    writer.add_scalar('Learning rate', scheduler.get_last_lr()[0], epoch_number + 1)
    writer.flush()

    # Track best performance and save model
    if avg_val_loss < best_vloss:
        best_vloss = avg_val_loss
        model_path = f'models/model_{epoch_number + 1}.pth'  # Save with dynamic name including the epoch number
        torch.save(alexnet.state_dict(), model_path)  # Save model

    epoch_number += 1

EPOCH 1:


 20%|██        | 1001/5005 [02:21<08:07,  8.21it/s]

  Batch 1000 Loss: 6.9087 Top-1 error rate: 99.90% Top-5 error rate: 99.47%


 40%|███▉      | 2001/5005 [04:40<08:57,  5.58it/s]

  Batch 2000 Loss: 6.7771 Top-1 error rate: 99.72% Top-5 error rate: 98.72%


...

  Validation Batch   300 Loss: 2.0040 Top-1 error: 44.89% Top-5 error: 22.38%


Validation: 100%|██████████| 391/391 [01:29<00:00,  4.39it/s]


LOSS train 1.0382 valid 1.7092
Top-1 Error train 27.64% val 39.98%
Top-5 Error train 8.34% val 17.71%
EPOCH 101:


  1%|          | 33/5005 [00:08<20:11,  4.10it/s] 
Exception in thread Thread-7 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 355, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_conn

KeyboardInterrupt: 